In [ ]:
from pathlib import Path
import os
import pandas as pd
import numpy as np

repo_path = Path('/home/krajda/anticipatio/')

In [ ]:
def open_fn(f):
    try:
        return pd.read_csv(f, engine='python')
    except:
        return pd.DataFrame()

tweets = pd.concat([
    pd.concat(map(open_fn, Path(repo_path / 'data/futurists_kol/data').rglob('*csv'))), 
    pd.concat(map(open_fn, Path(repo_path / 'data/futurists_rossdawson/data').rglob('*csv')))
])

tweets.columns = ['index','user','timestamp','url','txt']
tweets.reset_index(drop=True,inplace=True)

tweets['txt'] = tweets['txt'].astype(str)
tweets['timestamp'] = pd.to_datetime(tweets['timestamp'])
tweets = tweets.drop(columns=['index'])

tweets = tweets.drop_duplicates(subset=['txt'])

# TEXT CLEANING

In [ ]:
import html
import re
from tqdm import tqdm

regexes = [
    re.compile(
        r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,"
        r"}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|("
        r"?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    ), # URLS
    re.compile(r"\S*@\S*\..\S*"), # EMAILS
    re.compile(r"(?<=\s)(@[\w\-\.]+)(?=[\:\,\.\!\?\s]?)|^(@[\w\-\.]+)(?=[\:\,\.\!\?\s]?)"), # HANDLES
]

docs = tweets['txt'].tolist()

print('Cleaning tweets... html unescape')
docs = [html.unescape(t) for t in tqdm(docs)]

for regex in regexes:
    print('Cleaning tweets... removing regex #', regexes.index(regex))
    docs = [regex.sub('', t) for t in tqdm(docs)]

print('Cleaning tweets... removing RTs')
docs = [t[4:] if t.startswith('RT :') else t for t in tqdm(docs)]

tweets['original_text'] = tweets['txt']
tweets['txt'] = docs

In [ ]:
pd.to_pickle(tweets,repo_path / 'data/final.pkl')